[코드]
- 추천시스템 적용을 위해 원본 데이터를 변환하는 코드 

### 기본 라이브러리 Import 

In [ ]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split 

### 데이터 읽어오기 

In [ ]:
dog_reviews = pd.read_csv('../dataset/review_data_20220316/dogpre.gd_goods_review_202203160949.csv')
dog_goods = pd.read_csv('../dataset/review_data_20220316/dogpre.gd_goods_202203161103.csv')
dog_link = pd.read_csv('../dataset/review_data_20220316/dogpre.gd_goods_link_202203161106.csv')
dog_category = pd.read_csv('../dataset/review_data_20220316/dogpre.gd_category_202203161107.csv')
dog_order = pd.read_csv('../dataset/review_data_20220316/dogpre.gd_order_202203161029.csv')
dog_order_item = pd.read_csv('../dataset/review_data_20220316/dogpre.gd_order_item_202203161055.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4,15,17,26,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,4,5,23,29,32,52,53,54,55,66,74,78,81,95,103,121,122,131,141,147) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (13,17,29,40) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (17,39,41,42,43,44,47) have mixed types.Specify dtype option on im

### 데이터 전처리 

- 필요없는 컬럼 제거 
- Null 데이터 처리 
- 무결성 해치는 값들 제거 
- 중복 데이터 (후기) 제거 

In [ ]:
dog_goods.columns

Index(['goodsno', 'companyno', 'goodsnm', 'goodsnm_view', 'goodscd', 'barcode',
       'goods_type', 'origin_no', 'maker_no', 'brandno',
       ...
       'saturday_close_time', 'company_delivery_info_type', 'total_quantity',
       'unit_quantity', 'unit', 'live_mov_url', 'live_mov_title',
       'live_mov_hide', 'live_mov_thumbnail', 'hot_deal_thumbnail'],
      dtype='object', length=152)

#### 컬럼 재구성 

In [ ]:
r_cols = ['sno', 'goodsno', 'point', 'regdt', 'm_no', 'contents', 'pet_age', 'sex', 'dogname', 'recomm']
g_cols = ['goodsno', 'goodsnm', 'origin_no', 'maker_no', 'brandno', 'goods_type'] 
l_cols = ['sno', 'goodsno', 'category'] 
c_cols = ['sno' ,'category', 'catnm']
o_cols = ['ordno', 'settlekind', 'prn_settleprice', 'delivery', 'deliveryno', 'emoney']
oi_cols = ['sno', 'ordno', 'event_sno', 'event_type', 'event_goods_name','goodsno', 'goodsnm', 'ccode']

In [ ]:
dog_reviews = dog_reviews[r_cols]
dog_goods = dog_goods[g_cols]
dog_link = dog_link[l_cols]
dog_category = dog_category[c_cols] 
dog_order = dog_order[o_cols]
dog_order_item = dog_order_item[oi_cols]

In [ ]:
dog_reviews.head(3)

,sno,goodsno,point,regdt,m_no,contents,pet_age,sex,dogname,recomm
0,15,37634,5.0,2007-08-11 13:14:57.000,1,이거 병원에서 1000원이던데~!\r\n 이게 원래는 사료라던데!~? 강쥐들이 죠아...,NaN,남,시츄,0
1,17,37634,5.0,2007-08-27 13:16:45.000,1,이거 집 앞 슈퍼에서 750원에 팝니당~\r\n양이 더 많을줄 알았는데 같네요\r\...,NaN,여,슈나우져(스탠다드),0
2,19,37634,5.0,2007-09-06 13:18:17.000,1,사료줄때 ...몇개 넣어서주면 좋아요 ~^^...그래서 이번에 구매할때 이고 살려구...,NaN,여,푸들,6


In [ ]:
dog_order_item.head(3)

,sno,ordno,event_sno,event_type,event_goods_name,goodsno,goodsnm,ccode
0,2,1421442197805,315,BUNDLE,NaN,12829,도기세븐 미니 닭갈비 400g,0
1,4,1421442223148,2637,FREE_GIFT,NaN,15124,이름표 목걸이 줄 소형줄,0
2,6,1421442223148,2733,DATE,NaN,24260,엔디츄 소프트츄 치킨 2개입,0


#### 결측값 처리

- dog_reviews 테이블 결측값 처리 

In [ ]:
dog_reviews.isnull().sum()

sno               0
goodsno           0
point             0
regdt             0
m_no              0
contents        271
pet_age     1711670
sex            4800
dogname           0
recomm            0
dtype: int64

In [ ]:
len(dog_reviews)

4490421

In [ ]:
# 반려동물 결측 값 존재 행 삭제 
dog_reviews = dog_reviews.dropna()

In [ ]:
len(dog_reviews)

2778373

In [ ]:
# 반려동물 평균 나이 구하기 
mean = round(np.mean(dog_reviews['pet_age']), 1)

mean

5.5

- goods 테이블 결측값 처리 

In [ ]:
dog_goods.isnull().sum()

goodsno        0
goodsnm       19
origin_no      0
maker_no       0
brandno        0
goods_type     0
dtype: int64

In [ ]:
dog_goods = dog_goods.dropna()

- dog_order 테이블 결측값 처리 

In [ ]:
dog_order.isnull().sum()

ordno               0
settlekind         92
prn_settleprice     0
delivery            0
deliveryno          0
emoney              0
dtype: int64

In [ ]:
dog_order = dog_order.dropna()

- dog_order_item 테이블 결측값 처리 

In [ ]:
dog_order_item.isnull().sum()

sno                        0
ordno                      0
event_sno                  0
event_type          10759108
event_goods_name    16192010
goodsno                    0
goodsnm                    0
ccode                      0
dtype: int64

In [ ]:
dog_order_item['event_type'].unique()

array(['BUNDLE', 'FREE_GIFT', 'DATE', 'SELF_GIFT', nan, 'LIMIT',
       'STICKER', 'SUBOPT', 'COMBO', 'THEME', 'RECOMMEND', 'NEW',
       'BUNCHSALE', 'ONLY_DISPLAY'], dtype=object)

In [ ]:
dog_order_item['event_type'] = dog_order_item['event_type'].fillna(' ')
dog_order_item['event_goods_name'] = dog_order_item['event_goods_name'].fillna(' ')

#### 이상치 처리 

In [ ]:
filter = (dog_reviews['pet_age'] < 0)   
filter2 = (dog_reviews['pet_age'] > 20)
drop_idx = dog_reviews[filter].index   
drop_idx2 = dog_reviews[filter2].index   

dog_reviews = dog_reviews.drop(drop_idx)   # 필터에 해당하는 값들 제거 
dog_reviews = dog_reviews.drop(drop_idx2)   # 필터에 해당하는 값들 제거

In [ ]:
dog_reviews['point'].value_counts().head()

5.0    1838401
4.0     586166
3.0     244764
2.0      57694
1.0      33038
Name: point, dtype: int64

#### 무결성 해치는 데이터 제거 

In [ ]:
goods_item = dog_goods['goodsno'].unique()
reviews_item = dog_reviews['goodsno'].unique()

len(goods_item), len(reviews_item)
goods_item

array([    4,     9,    11, ..., 71671, 20438, 45284], dtype=int64)

In [ ]:
filter = (~dog_reviews['goodsno'].isin(goods_item))
drop_idx = dog_reviews[filter].index

dog_reviews = dog_reviews.drop(drop_idx)
len(dog_reviews)
# len(drop_idx)  # 12729

2760055

In [ ]:
dog_category.head()

,sno,category,catnm
0,128,16,사은품
1,356,51,간식
2,357,51001,수제간식
3,358,51002,껌
4,359,51003,저키/트릿


In [ ]:
filter = dog_link['goodsno'] == 37634

dog_link[filter]

,sno,goodsno,category
23054,79343,37634,51003


In [ ]:
len(dog_link['category'].unique())

414

In [ ]:
len(dog_reviews), len(dog_link), len(dog_category)

(2760055, 67603, 127)

In [ ]:
dog_reviews.head()

,sno,goodsno,point,regdt,m_no,contents,pet_age,sex,dogname,recomm
223972,239156,259,5.0,2010-04-26 21:45:48.000,29471,울 강쥐가 멋쟁이가 됐어요.사람들이 다 웃구 신기해 해여.,2.0,남,푸들,0
223992,239196,10459,5.0,2010-04-26 22:18:29.000,29471,"가끔 실수 하는데..,향이좋아요",2.0,남,푸들,0
223995,239203,8875,5.0,2010-04-26 22:20:58.000,29471,토이푸들인데 분홍색이랑 넘 잘어울려요,2.0,남,푸들,0
227509,239155,133,5.0,2010-04-26 21:44:26.000,29471,"사료를 바꿨는데,울 강쥐둘이 넘 잘먹어요.",2.0,남,푸들,0
228718,239195,256,5.0,2010-04-26 22:17:25.000,29471,울 강쥐가 신발만 봄 산책가자구 막 졸라요.산책갔다와두 발이 깨끗해서 좋아요,2.0,남,푸들,0


### 테이블 생성 

In [ ]:
dog_reviews = pd.merge(dog_reviews, dog_link, on='goodsno')

In [ ]:
dog_reviews = pd.merge(dog_reviews, dog_category, on='category')

In [ ]:
dog_reviews.columns

Index(['sno_x', 'goodsno', 'point', 'regdt', 'm_no', 'contents', 'pet_age',
       'sex', 'dogname', 'recomm', 'sno_y', 'category', 'sno', 'catnm'],
      dtype='object')

In [ ]:
r_cols = ['sno_x', 'goodsno', 'point', 'regdt', 'm_no', 'contents', 'pet_age', 'sex', 'dogname', 'catnm', 'recomm']
col_name = ['sno', 'goodsno', 'point', 'regdt', 'm_no', 'contents', 'pet_age', 'sex', 'dogname', 'catnm', 'recomm']
dog_reviews = dog_reviews[r_cols]
dog_reviews.columns = col_name

In [ ]:
dog_reviews.head(1)

,sno,goodsno,point,regdt,m_no,contents,pet_age,sex,dogname,catnm,recomm
0,239156,259,5.0,2010-04-26 21:45:48.000,29471,울 강쥐가 멋쟁이가 됐어요.사람들이 다 웃구 신기해 해여.,2.0,남,푸들,신발/양말,0


In [ ]:
df= dog_reviews.copy()

In [ ]:
df

,sno,goodsno,point,regdt,m_no,contents,pet_age,sex,dogname,catnm,recomm
0,239156,259,5.0,2010-04-26 21:45:48.000,29471,울 강쥐가 멋쟁이가 됐어요.사람들이 다 웃구 신기해 해여.,2.0,남,푸들,신발/양말,0
1,11262077,259,4.0,2015-01-27 20:49:33.000,262432,우리강아지 적응하는데는 좀 걸릴듯하지만\r\n신기면 귀여운행돌들을,7.0,남,푸들,신발/양말,0
2,11447447,259,4.0,2015-02-24 18:24:01.000,294441,저희강아지 2.4키로 토이푸들이구요 젤작은사이즈삿는데 완전 딱!!!!! 맞네요...,3.0,여,푸들,신발/양말,0
3,11475530,259,5.0,2015-03-08 13:35:17.000,213530,강아지가 편하게 신고 잘 다닙니다~~,4.0,남,비숑 프리제,신발/양말,0
4,11600282,259,3.0,2015-03-23 01:11:09.000,61889,좋아요 ..근데 좋긴한데 ㅠㅠ 밑창이 금방 닳아버렷어요.. 같은걸로 하나 더있는데 ...,1.0,남,말티즈,신발/양말,0
...,...,...,...,...,...,...,...,...,...,...,...
2754569,18525556,83477,5.0,2021-02-25 08:50:06.000,137605,앙증맞고 귀엽네요. 간식도 잘 먹어요.,15.0,남,말티즈,프로모션 사은품,0
2754570,18555358,83477,5.0,2021-03-10 16:52:04.000,1155249,또 구매합니다 ^.^!!!! 다른 맛도 사볼게요,12.0,여,푸들,프로모션 사은품,0
2754571,18866600,83477,5.0,2021-04-01 15:07:36.000,1158478,"가방에 배변봉투,물티슈,손수건 넣었는데도 공간이 두둑히 남아서 좋아요!\n\n간식도...",3.0,남,포메라니안,프로모션 사은품,0
2754572,18867941,83477,5.0,2021-04-02 13:53:16.000,576203,바삭츄좋아요..에코백산책시사용하니편하네요,6.0,남,기타견종 (중형견),프로모션 사은품,0


In [ ]:
df = df.drop_duplicates(['m_no', 'goodsno'])

df

,sno,goodsno,point,regdt,m_no,contents,pet_age,sex,dogname,catnm,recomm
0,239156,259,5.0,2010-04-26 21:45:48.000,29471,울 강쥐가 멋쟁이가 됐어요.사람들이 다 웃구 신기해 해여.,2.0,남,푸들,신발/양말,0
1,11262077,259,4.0,2015-01-27 20:49:33.000,262432,우리강아지 적응하는데는 좀 걸릴듯하지만\r\n신기면 귀여운행돌들을,7.0,남,푸들,신발/양말,0
2,11447447,259,4.0,2015-02-24 18:24:01.000,294441,저희강아지 2.4키로 토이푸들이구요 젤작은사이즈삿는데 완전 딱!!!!! 맞네요...,3.0,여,푸들,신발/양말,0
3,11475530,259,5.0,2015-03-08 13:35:17.000,213530,강아지가 편하게 신고 잘 다닙니다~~,4.0,남,비숑 프리제,신발/양말,0
4,11600282,259,3.0,2015-03-23 01:11:09.000,61889,좋아요 ..근데 좋긴한데 ㅠㅠ 밑창이 금방 닳아버렷어요.. 같은걸로 하나 더있는데 ...,1.0,남,말티즈,신발/양말,0
...,...,...,...,...,...,...,...,...,...,...,...
2754569,18525556,83477,5.0,2021-02-25 08:50:06.000,137605,앙증맞고 귀엽네요. 간식도 잘 먹어요.,15.0,남,말티즈,프로모션 사은품,0
2754570,18555358,83477,5.0,2021-03-10 16:52:04.000,1155249,또 구매합니다 ^.^!!!! 다른 맛도 사볼게요,12.0,여,푸들,프로모션 사은품,0
2754571,18866600,83477,5.0,2021-04-01 15:07:36.000,1158478,"가방에 배변봉투,물티슈,손수건 넣었는데도 공간이 두둑히 남아서 좋아요!\n\n간식도...",3.0,남,포메라니안,프로모션 사은품,0
2754572,18867941,83477,5.0,2021-04-02 13:53:16.000,576203,바삭츄좋아요..에코백산책시사용하니편하네요,6.0,남,기타견종 (중형견),프로모션 사은품,0


In [ ]:
len(df['m_no'].unique()), len(dog_reviews)

(88590, 2754574)

In [ ]:
m_count = pd.DataFrame(df['m_no'].value_counts())
m_count.columns=['m_no']
m_count

,m_no
709016,1226
190213,605
261571,592
11150,522
275152,511
...,...
903753,1
981994,1
764549,1
190775,1


In [ ]:
m_count = m_count[m_count['m_no'] > 1]

len(m_count)

82020

In [ ]:
user_list = pd.DataFrame(m_count.index)

user_list.columns=['m_no']
user_list

,m_no
0,709016
1,190213
2,261571
3,11150
4,275152
...,...
82015,1125433
82016,857454
82017,65571
82018,1163840


In [ ]:
user_list[user_list['m_no']==1119862]

,m_no


In [ ]:
df2 = user_list.copy()

In [ ]:
dog_reviews[dog_reviews['m_no'] == 709016]

,sno,goodsno,point,regdt,m_no,contents,pet_age,sex,dogname,catnm,recomm
3311,14770868,10770,5.0,2017-06-22 11:18:03.000,709016,잘 받았어요.,4.0,남,푸들,신발/양말,0
10759,14770898,11357,5.0,2017-06-22 11:19:29.000,709016,잘 받았어요.,4.0,남,푸들,탈취/소독,0
15332,14770880,15035,5.0,2017-06-22 11:18:37.000,709016,잘 받았어요.,4.0,남,푸들,탈취/소독,0
17561,14770841,10456,5.0,2017-06-22 11:16:57.000,709016,향이 좋아요.,4.0,남,푸들,탈취/소독,0
18534,14804450,21908,5.0,2017-07-01 11:19:49.000,709016,향이 나는 탈취제를 쓰고 있는데..\r\n여름이 되니 좀 더 위생에 신경을 써야 할...,4.0,남,푸들,탈취/소독,0
...,...,...,...,...,...,...,...,...,...,...,...
2753266,14912309,48071,5.0,2017-07-27 15:59:43.000,709016,잘 받았어요~,4.0,남,푸들,구강티슈,0
2753639,18866890,82204,5.0,2020-12-29 14:58:11.000,709016,오자마자 먹여봤는데 잘 먹네요 ㅎ,7.0,남,포메라니안,사은품,0
2753670,17838826,60862,5.0,2020-01-20 17:20:21.000,709016,배변판 바로 위쪽에 설치했는데..아직 효과는 잘 모르겠어요,7.0,남,포메라니안,공기청정,0
2753691,17838835,60859,5.0,2020-01-20 17:20:49.000,709016,빠른 배송 감사합니다~!,7.0,남,포메라니안,공기청정,0


In [ ]:
user = pd.merge(df, df2, on='m_no')

user

,sno,goodsno,point,regdt,m_no,contents,pet_age,sex,dogname,catnm,recomm
0,239156,259,5.0,2010-04-26 21:45:48.000,29471,울 강쥐가 멋쟁이가 됐어요.사람들이 다 웃구 신기해 해여.,2.0,남,푸들,신발/양말,0
1,239195,256,5.0,2010-04-26 22:17:25.000,29471,울 강쥐가 신발만 봄 산책가자구 막 졸라요.산책갔다와두 발이 깨끗해서 좋아요,2.0,남,푸들,신발/양말,0
2,239204,258,5.0,2010-04-26 22:22:00.000,29471,맨 처음엔 신구 잘못걸었는데 지금은 잘걸어요,2.0,남,푸들,신발/양말,0
3,14171738,35564,3.0,2017-01-29 15:07:09.000,29471,천으로 되어있어 얼마안신고 구멍이 났어요~ㅠ\n,9.0,남,푸들,신발/양말,0
4,15968097,3042,5.0,2009-04-26 22:17:25.000,29471,울 강쥐가 신발만 봄 산책가자구 막 졸라요.산책갔다와두 발이 깨끗해서 좋아요,1.0,남,푸들,신발/양말,0
...,...,...,...,...,...,...,...,...,...,...,...
2455804,18978469,57527,5.0,2021-07-20 14:31:01.000,1160250,아직 급여전인데 잘먹엇우면 좋겟네요 ㅎ,3.0,남,푸들,건조생식,0
2455805,18982473,66361,5.0,2021-07-22 01:33:16.000,1160250,잘묵네요 ㅋ3개사면 하나더준데서 요걸로 바꿧더니 너무잘먹어서 다먹으면 또 재구매할거에요,3.0,남,푸들,건조생식,0
2455806,18429535,63154,5.0,2020-10-23 11:13:26.000,1152011,빠른배송 감사합니다 ^^,8.0,남,시바 이누,건조생식,0
2455807,18452630,63148,5.0,2020-11-24 23:18:00.000,1152011,잘먹일게요또주문할게용,8.0,남,시바 이누,건조생식,0


#### ratings

In [ ]:
rating_cols = ['m_no', 'goodsno', 'point', 'regdt', 'pet_age', 'dogname']

In [ ]:
dog_ratings = user[rating_cols]

dog_ratings

,m_no,goodsno,point,regdt,pet_age,dogname
0,29471,259,5.0,2010-04-26 21:45:48.000,2.0,푸들
1,29471,256,5.0,2010-04-26 22:17:25.000,2.0,푸들
2,29471,258,5.0,2010-04-26 22:22:00.000,2.0,푸들
3,29471,35564,3.0,2017-01-29 15:07:09.000,9.0,푸들
4,29471,3042,5.0,2009-04-26 22:17:25.000,1.0,푸들
...,...,...,...,...,...,...
2455804,1160250,57527,5.0,2021-07-20 14:31:01.000,3.0,푸들
2455805,1160250,66361,5.0,2021-07-22 01:33:16.000,3.0,푸들
2455806,1152011,63154,5.0,2020-10-23 11:13:26.000,8.0,시바 이누
2455807,1152011,63148,5.0,2020-11-24 23:18:00.000,8.0,시바 이누


In [ ]:
df= dog_ratings.copy()

In [ ]:
df = df.drop_duplicates(['m_no', 'goodsno'])

df

,m_no,goodsno,point,regdt,pet_age,dogname
0,29471,259,5.0,2010-04-26 21:45:48.000,2.0,푸들
1,29471,256,5.0,2010-04-26 22:17:25.000,2.0,푸들
2,29471,258,5.0,2010-04-26 22:22:00.000,2.0,푸들
3,29471,35564,3.0,2017-01-29 15:07:09.000,9.0,푸들
4,29471,3042,5.0,2009-04-26 22:17:25.000,1.0,푸들
...,...,...,...,...,...,...
2455804,1160250,57527,5.0,2021-07-20 14:31:01.000,3.0,푸들
2455805,1160250,66361,5.0,2021-07-22 01:33:16.000,3.0,푸들
2455806,1152011,63154,5.0,2020-10-23 11:13:26.000,8.0,시바 이누
2455807,1152011,63148,5.0,2020-11-24 23:18:00.000,8.0,시바 이누


In [ ]:
df['m_no'].value_counts()

709016     1226
190213      605
261571      592
11150       522
275152      511
           ... 
1018021       2
946684        2
1163125       2
977614        2
1217436       2
Name: m_no, Length: 82020, dtype: int64

In [ ]:
user_cols = ['m_no', 'dogname', 'pet_age']

user = df[user_cols]
user = user.drop_duplicates(['m_no'])
user

,m_no,dogname,pet_age
0,29471,푸들,2.0
91,262432,푸들,7.0
94,294441,푸들,3.0
102,213530,비숑 프리제,4.0
163,61889,말티즈,1.0
...,...,...,...
2455796,1160571,치와와,3.0
2455798,1164900,셔틀랜드 쉽독,5.0
2455801,1193954,비숑 프리제,3.0
2455804,1160250,푸들,3.0


### 각각의 테이블 CSV 파일로 저장 

#### 원본 테이블 생성 

In [ ]:
df.to_csv('./dog_ratings.csv', index=None)
user.to_csv('./dog_user.csv', index=None)

#### 10만개 테이블 생성

In [ ]:
dog_rating_100000 = dog_ratings.copy() 
dog_rating_100000 = dog_rating_100000[:100000]

In [ ]:
dog_rating_100000.to_csv('./dog_data.csv', index=None)